# Recommeded Systems - Project

### 1. Import the necessary libraries and read the provided CSVs as a data frame and perform the below steps. [15 Marks]

In [1]:
#Importing all the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import glob
from surprise import SVD, Reader, Dataset, dataset, KNNWithMeans, accuracy
from surprise.model_selection import cross_validate, train_test_split

import collections

#### 1A. Merge all the provided CSVs into one dataFrame. [2 Marks]

In [2]:
#Reading all the csv files

#Providing the commom path of all the csv files
path = 'D:/Great Learning/Study Material/8. Module-7_Recommendation Systems/Recommended Systems - Project/Data Set/Data Set'
csv_file = glob.glob(path + '/*.csv')

#Creating a function that can use the parameters of read_csv
def readcsv(args):
    return (pd.read_csv(args, encoding='latin-1'))

#Listing all the csv files & concatnating them into 1 dataframe
df_list = (readcsv(file) for file in csv_file)
phones = pd.concat(df_list, ignore_index=True)

In [3]:
#Displaying the top 5 rows of the dataset
phones.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


#### 1B. Explore, understand the Data and share at least 2 observations. [2 Marks]

In [4]:
#Exploring the dataset
phones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 118.8+ MB


In [5]:
#Using the describe method on phones dataset
phones.describe()

,score,score_max
count,1.351644e+06,1351644.0
mean,8.007060e+00,10.0
std,2.616121e+00,0.0
min,2.000000e-01,10.0
25%,7.200000e+00,10.0
50%,9.200000e+00,10.0
75%,1.000000e+01,10.0
max,1.000000e+01,10.0


In [6]:
#Changing the 'date' column datatype to datetime format from object datatype
phones['date'] = pd.to_datetime(phones['date'], infer_datetime_format=True)
phones.info()
phones.head()

C:\Users\smit1\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '17/11/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\smit1\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\smit1\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '27/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\smit1\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '26/10/2015' in DD/MM/YYYY format. Provide format or specif

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   phone_url  1415133 non-null  object        
 1   date       1415133 non-null  datetime64[ns]
 2   lang       1415133 non-null  object        
 3   country    1415133 non-null  object        
 4   source     1415133 non-null  object        
 5   domain     1415133 non-null  object        
 6   score      1351644 non-null  float64       
 7   score_max  1351644 non-null  float64       
 8   extract    1395772 non-null  object        
 9   author     1351931 non-null  object        
 10  product    1415132 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 118.8+ MB


,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,2017-05-02,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,2017-04-28,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,2017-05-04,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,2017-05-02,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,2017-05-11,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [7]:
#Removing Non-Ascii characters & white space from the 'author' & 'product' column
cols = ['author', 'product']
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–","$",".","","¨"]
for col in cols:
    for char in spec_chars:
        phones[col] = phones[col].str.replace(char, ' ')
    phones[col].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
    phones[col].replace(r'^\s+$', np.nan, regex=True, inplace=True)
    phones[col].replace('', np.nan, regex=True, inplace=True)

C:\Users\smit1\AppData\Local\Temp\ipykernel_10192\2106209644.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  phones[col] = phones[col].str.replace(char, ' ')


##### Observations:
- There seems to be multiple null values present in the dataset
- The feature 'score_max' has all the values as 10. Hence it is practically incompetent for any recommendation.
- The 'date' feature needed to be converted to proper format. Hence we changed it to datetime format from object format.
- The minimum rating received is 0.2 while the maximum is 10.

#### 1C. Round off scores to the nearest integers. [3 Marks]

In [8]:
#Using the round() function to convert the score to nearest integer
#Using round() converts the float to integer as per proper format i.e. if an odd number before 0.5 should be converted to
#even number, while an even number should remain same
#i.e 1.5 should be rounded off to 2, while 2.5 should be rounded off to 2. This features is supported by round() function

#Rounding off the score to nearest integer
phones['score'] = phones['score'].round(0).astype('Int64')
phones['score'] = pd.to_numeric(phones['score'], errors='ignore', downcast='integer')
phones['score']

0          10
1          10
2           6
3           9
4           4
           ..
1415128     2
1415129    10
1415130     2
1415131     8
1415132     2
Name: score, Length: 1415133, dtype: Int8

In [9]:
phones['score'].unique()

<IntegerArray>
[10, 6, 9, 4, 8, 2, 7, 5, <NA>, 3, 1, 0]
Length: 12, dtype: Int8

#### 1D. Check for missing values. Impute the missing values, if any. [2 Marks]

In [10]:
#Checking for missing/null values
Nulls = pd.DataFrame({'Null_val': phones.isnull().sum(), 'Null(%)': round(phones.isnull().sum()/len(phones)*100,2)})
Nulls

,Null_val,Null(%)
phone_url,0,0.00
date,0,0.00
lang,0,0.00
country,0,0.00
source,0,0.00
domain,0,0.00
score,63489,4.49
score_max,63489,4.49
extract,19361,1.37
author,133824,9.46


- Here, the 'score' feature is of the prime importance, & further it is numerical type hence it can ne imputed. We use mean of the 'score' column to fill the null values.
- The 'score_max' feature represents the highest score given for product which is 10 & as mentioned earlier it is of no use to us.
- The 'extract' feature also does not play a significant role in recommendations. Hence even if there are any null values, they can be dealt later on.
- The 'author' feature represents the user. In this case, there are multiple null values present. Imputing them with other values like 'Unknown' / 'Invalid_User' shall not give a proper justification as to judge the user characteristics. Hence it will be better to drop the the data with null 'author'.
- Further 'product' columns also shows 1 null value, which can be dropped.

- We initially impute the 'score' with the mean value & then drop the null values of 'author' feature. If we drop the null values first & then impute the 'score' with mean, the mean value shall change due to reduced data. Hence to get the exact mean of the dataset we impute the 'score' prior to dropping null values.

In [11]:
#Imputing the 'score' feature with its mean value. For 'score_max' feature this is not necessary as all the values are same
#& hence the mean value is also same. But we impute for 'score_max' feature as well.

#Since we have rounded of all the scores to its nearest integer, we take the integer value for mean as well
mean_fill = lambda x: x.fillna(round(x.mean(), 0))
phones[['score', 'score_max']] = phones[['score', 'score_max']].apply(mean_fill, axis=0)

In [12]:
#Dropping the null values of 'author' column
phones.drop(phones[phones['author'].isnull()==True].index, axis=0, inplace=True)

In [13]:
#Checking the null values again after imputing & dropping values
phones.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score            0
score_max        0
extract      11987
author           0
product          0
dtype: int64

- Here we see that only 'extract' feature now has null values present in it.
- As of now we don't deal with it.
- Later on during removal of the irrelevant feature we drop this feature. Hence the null values in it doesnot matter much.
- Further 1 null value that was present in the 'product' column, has been dropped during the process of dropping the null values in 'author' column. It means that one particular row did not have value at author as well as at product place.

#### 1E. Check for duplicate values and remove them, if any. [2 Marks]

In [14]:
#Checking & dropping the duplicate rows if any
duplicate = phones.duplicated()
print(sum(duplicate))
phones.shape

7886


(1281309, 11)

In [15]:
#Dropping the duplicate data
phones = phones.drop_duplicates()
phones.shape

(1273423, 11)

#### 1F. Keep only 1 Million data samples. Use random state=612. [2 Marks]

In [16]:
#Creating New dataframe with only 1000000 records.
phones_1m = phones.sample(n=1000000, random_state=612)
phones_1m.shape

(1000000, 11)

#### 1G. Drop irrelevant features. Keep features like Author, Product, and Score. [2 Marks]

In [17]:
#Dropping irrelevant features
mobile = phones.drop(['phone_url', 'domain', 'score_max', 'extract'], axis=1)
mobile.shape

(1273423, 7)

### 2. Answer the following questions. [10 Marks]

#### 2A. Identify the most rated products. [3 Marks]

In [18]:
#Here in the dataset all the products have been rated
#Identifying most rated products
most_rated_product = pd.DataFrame({'Product_Name': mobile['product'].value_counts().keys(), 'Total_Ratings(nos.)': mobile['product'].value_counts().values})
most_rated_product.head()

,Product_Name,Total_Ratings(nos.)
0,Lenovo Vibe K4 Note White 16GB,5223
1,Lenovo Vibe K4 Note Black 16GB,4389
2,OnePlus 3 Graphite 64 GB,4103
3,OnePlus 3 Soft Gold 64 GB,3557
4,Huawei P8lite zwart 16 GB,2707


- The mobile 'Lenovo Vibe K4 Note(White, 16GB)' is the most rated product with 3847 no. of ratings.

#### 2B. Identify the users with most number of reviews. [3 Marks]

In [19]:
#Identifying the users who have rated the most times.
user_most_rated = pd.DataFrame({'User': mobile['author'].value_counts().keys(), 'Total_Ratings_provided': mobile['author'].value_counts().values})
user_most_rated.head()

,User,Total_Ratings_provided
0,Amazon Customer,76731
1,Cliente Amazon,19304
2,e bit,8411
3,Client d Amazon,7716
4,Amazon Kunde,5282


- Here we observe that the people using amazon services have rated the most as anonymous users.

#### 2C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final dataset. [4 Marks]

In [20]:
#Creating dataframe with products rated with score above 50 in total.
product_above50 = mobile.groupby('product')['score'].sum().sort_values(ascending=False).reset_index()[mobile.groupby('product')['score'].sum().sort_values(ascending=False).reset_index()['score']>50]
product_above50

,product,score
0,Lenovo Vibe K4 Note White 16GB,37502
1,OnePlus 3 Graphite 64 GB,35800
2,Lenovo Vibe K4 Note Black 16GB,31484
3,OnePlus 3 Soft Gold 64 GB,30242
4,Huawei P8lite zwart 16 GB,22895
...,...,...
25692,Sony Ericsson BeJoo,51
25693,8890,51
25694,Galaxy S5 CDMA Galaxy S5 Sport,51
25695,LG P715 Optimus L7 II Dual Black,51


In [21]:
#Creating dataframe with user who have rated a total score above 50.
user_rated_above50 = mobile.groupby(['author'])['score'].sum().sort_values(ascending=False).reset_index()[mobile.groupby(['author'])['score'].sum().sort_values(ascending=False).reset_index()['score']>50]
user_rated_above50

,author,score
0,Amazon Customer,541478
1,Cliente Amazon,154326
2,e bit,74766
3,Client d Amazon,56104
4,Amazon Kunde,38740
...,...,...
10111,Qbanbull,51
10112,chirag,51
10113,guy guata,51
10114,Debo,51


In [22]:
#Creating a dataframe with products which have been rated above 50 in total & users who have rated a total above 50.
mob_50 = mobile[mobile['product'].isin(product_above50['product']) & mobile['author'].isin(user_rated_above50['author'])]
mob_50.reset_index(drop=True, inplace=True)
mob_50

,date,lang,country,source,score,author,product
0,2017-05-06,en,gb,Very,10,Moses,Samsung Galaxy S8 64Gb Black
1,2017-05-06,en,gb,Very,10,Anonymous,Samsung Galaxy S8 64Gb Black
2,2017-05-06,en,gb,Very,10,Droid,Samsung Galaxy S8 64Gb Black
3,2017-05-06,en,gb,Very,10,Shay,Samsung Galaxy S8 64Gb Orchid Grey
4,2017-05-06,en,gb,Very,10,James,Samsung Galaxy S8 64Gb Orchid Grey
...,...,...,...,...,...,...,...
340236,2000-01-06,de,de,Dooyoo,6,Ashman,Siemens C10 D
340237,2000-01-04,de,de,Dooyoo,8,speedybit,Siemens C10 D
340238,2000-01-03,de,de,Dooyoo,6,bankier1,Siemens C10 D
340239,1999-12-30,de,de,Dooyoo,2,Rockshox,Siemens C10 D


In [23]:
#Shape of final dataframe
mob_50.shape

(340241, 7)

- The shape of final dataset having products rated with total score above 50 & user who have rated a total score above 50 is 456995 rows & 7 columns.
- Hereafter we use this dataset for recommendations & predictions for new products & users.

### 3. Build a popularity based model and recommend top 5 mobile phones. [5 Marks]

In [24]:
#Popularity of a model can be defined by how many people are aware/used the product i.e. what is the spread of the product.
prod_author = mob_50.groupby(['product'])['author'].nunique().sort_values(ascending=False).reset_index()
prod_avgscore = mob_50.groupby('product')['score'].mean().sort_values(ascending=False).reset_index()
popular_models = pd.merge(prod_author, prod_avgscore, on='product')
popular_models

,product,author,score
0,Huawei P8lite zwart 16 GB,429,8.49061
1,OnePlus 3 Graphite 64 GB,330,8.594499
2,Huawei P9 Lite Smartphone LTE Display 5 2 ...,323,9.135458
3,Lenovo Vibe K4 Note White 16GB,312,7.015818
4,Lenovo Motorola Moto G Smartphone 4 5 pollici...,308,9.449139
...,...,...,...
22792,Samsung Galaxy S6 Edge 32gb Black 6750A,1,9.222222
22793,Motorola ROKR Z6 Unlocked Phone with 2 MP Came...,1,4.0
22794,HTC One M9 Gold,1,10.0
22795,Motorola Razr HD,1,10.0


In [25]:
#Recommending top5 models
Top_5 = popular_models.head()['product']
Top_5

0                          Huawei P8lite zwart   16 GB
1                          OnePlus 3  Graphite  64 GB 
2    Huawei P9 Lite Smartphone  LTE  Display 5 2   ...
3                     Lenovo Vibe K4 Note  White 16GB 
4    Lenovo Motorola Moto G Smartphone  4 5 pollici...
Name: product, dtype: object

### 4. Build a collaborative filtering model using SVD. You can use SVD from surprise or build it from scratch(Note: Incase you’re building it from scratch you can limit your data points to 5000 samples if you face memory issues). Build a collaborative filtering model using kNNWithMeans from surprise. You can try both user-based and item-based model. [10 Marks]

#### # Using SVD from surprise library to construct collaborative model

In [26]:
#Converting the pandas library dataframe into surprise library readable format

#Getting Min & Max rating as feeder to reader
min_rate = mob_50['score'].min()
max_rate = mob_50['score'].max()

#Provideing the reader
reader = Reader(rating_scale=(min_rate, max_rate))

#Creating Dataframe readable to surprise library
mobs = Dataset.load_from_df(mob_50[['author', 'product', 'score']], reader)


#Source:https://medium.com/tiket-com/get-to-know-with-surprise-2281dd227c3e

In [27]:
#Splitting the dataset into train & test set using suprise library inbuilt function for this colabrative model
trainset, testset = train_test_split(mobs, test_size=0.20, random_state=1)

In [28]:
#Building the SVD model
sur_svd = SVD(random_state=1)

#Fitting the model on 'train' set
sur_svd.fit(trainset)

In [29]:
#Predicting the testset
sur_svd_pred = sur_svd.test(testset)

In [30]:
sur_svd_pred

[Prediction(uid='Mal', iid='Sim Free Motorola Moto G 4th Generation Mobile Phone   Black', r_ui=10.0, est=9.033734736843536, details={'was_impossible': False}),
 Prediction(uid='lahr2006', iid='Samsung Galaxy Express I8730', r_ui=10.0, est=8.475565473701696, details={'was_impossible': False}),
 Prediction(uid='e bit', iid='Smartphone Motorola Moto G 4 Play XT1603', r_ui=10.0, est=8.886095794576928, details={'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Huawei Honor 5X  Grey  16GB ', r_ui=10.0, est=9.364705385774128, details={'was_impossible': False}),
 Prediction(uid='David', iid='AICEK Coque Samsung Galaxy A3 2016  AICEK Etui Silicone Gel Samsung Galaxy A3 2016  A310F  Housse Antichoc Samsung A3 Transparente Souple Coque De Protection Pour Samsung Galaxy A3 2016  4 7 pouces ', r_ui=8.0, est=8.039583254220783, details={'was_impossible': False}),
 Prediction(uid='Flavio', iid='Nokia Asha 200 Telefono Cellulare  Dual SIM  Display da 2 4   Nero', r_ui=10.0, est=8.2777

#### #Building Colabrative Filtering Model using KNNWithMeans

- Using the 'mobs' dataset which has already been converted to surprise library readable format from the original pandas library dataset 'mob_50'.

In [31]:
#Building the model using KNNWithMeans (Considering k=50)
#A.) User-Based
kwmup = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

#B.) Item-Based
kwmip = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [32]:
#Fitting the User-Based model with trainset
kwmup.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [33]:
#Predicting the values using 'kwmup' model of testset
kwmup_pred = kwmup.test(testset)

In [34]:
kwmup_pred

[Prediction(uid='Mal', iid='Sim Free Motorola Moto G 4th Generation Mobile Phone   Black', r_ui=10.0, est=10, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='lahr2006', iid='Samsung Galaxy Express I8730', r_ui=10.0, est=8.44, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='e bit', iid='Smartphone Motorola Moto G 4 Play XT1603', r_ui=10.0, est=8.76, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Huawei Honor 5X  Grey  16GB ', r_ui=10.0, est=7.880000000000002, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='David', iid='AICEK Coque Samsung Galaxy A3 2016  AICEK Etui Silicone Gel Samsung Galaxy A3 2016  A310F  Housse Antichoc Samsung A3 Transparente Souple Coque De Protection Pour Samsung Galaxy A3 2016  4 7 pouces ', r_ui=8.0, est=8.282623493602856, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='Flavio', iid='Nokia Asha 200 Telefono Cellulare  Dual SIM  Displa

In [35]:
#Fitting the Item-Based model with trainset
kwmip.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [36]:
#Predicting the values using 'kwmip' model of testset
kwmip_pred = kwmip.test(testset)

In [37]:
kwmip_pred

[Prediction(uid='Mal', iid='Sim Free Motorola Moto G 4th Generation Mobile Phone   Black', r_ui=10.0, est=9.838317423189704, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid='lahr2006', iid='Samsung Galaxy Express I8730', r_ui=10.0, est=8.44, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='e bit', iid='Smartphone Motorola Moto G 4 Play XT1603', r_ui=10.0, est=8.76, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Amazon Customer', iid='Huawei Honor 5X  Grey  16GB ', r_ui=10.0, est=7.88, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='David', iid='AICEK Coque Samsung Galaxy A3 2016  AICEK Etui Silicone Gel Samsung Galaxy A3 2016  A310F  Housse Antichoc Samsung A3 Transparente Souple Coque De Protection Pour Samsung Galaxy A3 2016  4 7 pouces ', r_ui=8.0, est=8.13888063727026, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='Flavio', iid='Nokia Asha 200 Telefono Cellulare  Dual SIM  Displa

### 5. Evaluate the collaborative model. Print RMSE value. [2 Marks]

In [38]:
#Printing the RMSE values of all the colabrative models
Model_Accuracy = pd.DataFrame({'SVD_Model': accuracy.rmse(sur_svd_pred), 'UB_KWM_Model': accuracy.rmse(kwmup_pred), 
                               'IB_KWM_Model': accuracy.rmse(kwmip_pred)}, index=['Accuracy (RMSE)'])
Model_Accuracy

RMSE: 2.6414
RMSE: 2.7600
RMSE: 2.7278


,SVD_Model,UB_KWM_Model,IB_KWM_Model
Accuracy (RMSE),2.641431,2.760045,2.727831


### 6. Predict score (average rating) for test users. [2 Marks]

In [39]:
#Converting the output of 'SVD' model 'sur_svd_pred' to Dataframe
sur_svd_pred_df = pd.DataFrame(sur_svd_pred)
sur_svd_pred_df

,uid,iid,r_ui,est,details
0,Mal,Sim Free Motorola Moto G 4th Generation Mobile...,10.0,9.033735,{'was_impossible': False}
1,lahr2006,Samsung Galaxy Express I8730,10.0,8.475565,{'was_impossible': False}
2,e bit,Smartphone Motorola Moto G 4 Play XT1603,10.0,8.886096,{'was_impossible': False}
3,Amazon Customer,Huawei Honor 5X Grey 16GB,10.0,9.364705,{'was_impossible': False}
4,David,AICEK Coque Samsung Galaxy A3 2016 AICEK Etui...,8.0,8.039583,{'was_impossible': False}
...,...,...,...,...,...
68044,Giacomo,Samsung G900 Galaxy S5 Smartphone 16 GB Nero...,2.0,8.056695,{'was_impossible': False}
68045,Joey,Apple iPhone 6 Plus GSM Unlocked Cellphone 64...,4.0,8.368495,{'was_impossible': False}
68046,umoo,Samsung Omnia i900 Cep Telefonu,8.0,7.902566,{'was_impossible': False}
68047,Harry,Microsoft Nokia X2 02 Handy 5 6 cm 2 2 Zoll ...,10.0,7.685926,{'was_impossible': False}


In [40]:
#Converting the output of 'kwmup' model 'kwmup_pred' to DataFrame
kwmup_pred_df = pd.DataFrame(kwmup_pred)
kwmup_pred_df

,uid,iid,r_ui,est,details
0,Mal,Sim Free Motorola Moto G 4th Generation Mobile...,10.0,10.000000,"{'actual_k': 10, 'was_impossible': False}"
1,lahr2006,Samsung Galaxy Express I8730,10.0,8.440000,"{'actual_k': 50, 'was_impossible': False}"
2,e bit,Smartphone Motorola Moto G 4 Play XT1603,10.0,8.760000,"{'actual_k': 50, 'was_impossible': False}"
3,Amazon Customer,Huawei Honor 5X Grey 16GB,10.0,7.880000,"{'actual_k': 50, 'was_impossible': False}"
4,David,AICEK Coque Samsung Galaxy A3 2016 AICEK Etui...,8.0,8.282623,"{'actual_k': 14, 'was_impossible': False}"
...,...,...,...,...,...
68044,Giacomo,Samsung G900 Galaxy S5 Smartphone 16 GB Nero...,2.0,7.661668,"{'actual_k': 22, 'was_impossible': False}"
68045,Joey,Apple iPhone 6 Plus GSM Unlocked Cellphone 64...,4.0,8.422491,"{'actual_k': 50, 'was_impossible': False}"
68046,umoo,Samsung Omnia i900 Cep Telefonu,8.0,8.000000,"{'actual_k': 50, 'was_impossible': False}"
68047,Harry,Microsoft Nokia X2 02 Handy 5 6 cm 2 2 Zoll ...,10.0,9.445238,"{'actual_k': 1, 'was_impossible': False}"


In [41]:
#Converting the output of 'kwmip' model 'kwmip_pred' to DataFrame
kwmip_pred_df = pd.DataFrame(kwmip_pred)
kwmip_pred_df

,uid,iid,r_ui,est,details
0,Mal,Sim Free Motorola Moto G 4th Generation Mobile...,10.0,9.838317,"{'actual_k': 4, 'was_impossible': False}"
1,lahr2006,Samsung Galaxy Express I8730,10.0,8.440000,"{'actual_k': 50, 'was_impossible': False}"
2,e bit,Smartphone Motorola Moto G 4 Play XT1603,10.0,8.760000,"{'actual_k': 50, 'was_impossible': False}"
3,Amazon Customer,Huawei Honor 5X Grey 16GB,10.0,7.880000,"{'actual_k': 50, 'was_impossible': False}"
4,David,AICEK Coque Samsung Galaxy A3 2016 AICEK Etui...,8.0,8.138881,"{'actual_k': 50, 'was_impossible': False}"
...,...,...,...,...,...
68044,Giacomo,Samsung G900 Galaxy S5 Smartphone 16 GB Nero...,2.0,7.782960,"{'actual_k': 33, 'was_impossible': False}"
68045,Joey,Apple iPhone 6 Plus GSM Unlocked Cellphone 64...,4.0,6.663686,"{'actual_k': 12, 'was_impossible': False}"
68046,umoo,Samsung Omnia i900 Cep Telefonu,8.0,8.000000,"{'actual_k': 47, 'was_impossible': False}"
68047,Harry,Microsoft Nokia X2 02 Handy 5 6 cm 2 2 Zoll ...,10.0,5.648717,"{'actual_k': 11, 'was_impossible': False}"


In [42]:
#Converting the original 'testset' to DataFrame
testset_df = pd.DataFrame(testset, columns=['Author', 'Product', 'Actual_Avg_Ratings'])
testset_df

,Author,Product,Actual_Avg_Ratings
0,Mal,Sim Free Motorola Moto G 4th Generation Mobile...,10.0
1,lahr2006,Samsung Galaxy Express I8730,10.0
2,e bit,Smartphone Motorola Moto G 4 Play XT1603,10.0
3,Amazon Customer,Huawei Honor 5X Grey 16GB,10.0
4,David,AICEK Coque Samsung Galaxy A3 2016 AICEK Etui...,8.0
...,...,...,...
68044,Giacomo,Samsung G900 Galaxy S5 Smartphone 16 GB Nero...,2.0
68045,Joey,Apple iPhone 6 Plus GSM Unlocked Cellphone 64...,4.0
68046,umoo,Samsung Omnia i900 Cep Telefonu,8.0
68047,Harry,Microsoft Nokia X2 02 Handy 5 6 cm 2 2 Zoll ...,10.0


In [43]:
#Grouping all the dataframes by 'uid', averaging the acutal ratings by user from testset & averaging the estimated ratings
#by user from all other dataset (i.e. sur_svd_pred_df, kwmup_pred_df & kwmip_pred_df)
User_Avg_Rating = pd.DataFrame(testset_df.groupby('Author')['Actual_Avg_Ratings'].mean())
User_Avg_Rating['SVD_Pred_Avg_Ratings'] = sur_svd_pred_df.groupby('uid')['est'].mean().values
User_Avg_Rating['KMWUP_Pred_Avg_Ratings'] = kwmup_pred_df.groupby('uid')['est'].mean().values
User_Avg_Rating['KWMIP_Pred_Avg_Ratings'] = kwmip_pred_df.groupby('uid')['est'].mean().values
User_Avg_Rating

,Actual_Avg_Ratings,SVD_Pred_Avg_Ratings,KMWUP_Pred_Avg_Ratings,KWMIP_Pred_Avg_Ratings
Author,,,,
Pro Street,10.0,9.191312,9.748476,10.000000
Francis Mazzucco Spain Premium Top Reviewer,10.0,9.330202,10.000000,9.746758
mer,8.0,8.005681,8.000000,8.000000
1,8.0,7.065773,7.666667,6.562500
13,10.0,8.376140,9.500000,8.514286
...,...,...,...,...
ziofrankie,8.0,8.217582,8.666667,8.666667
zoe24,10.0,7.746583,7.983419,8.000000
zomtech,8.8,9.057014,8.728168,8.745087


### 7. Report your findings and inferences. [2 Marks]

##### Observations:
- From the above 3 models: SVD, KNNWithMeans(User-Based) & KNNWithMeans(Item-Based) we see that the least RMSE score is for SVD model.
- This can suggest that for this particular dataset, after data cleaning, SVD model predict the nearest possible value to the original values.
- However, a noticeable point is also that although the RMSE score of SVD model is least, the RMSE score for other models can be considered similar as they do not have significant difference.
- For proper representation, the dataframe 'User_Avg_Rating' is created to compare the average predicted ratings of all uesr model-wise.

### 8. Try and recommend top 5 products for test users. [5 Marks]

In [44]:
#Defining a function that returns top_5 products for a user
def top_rec(pred, n=5):
    top_n = collections.defaultdict(list)
    for uid, iid, actu, est, _ in pred:
        top_n[uid].append((iid, est))
    
    for uid, recom_rate in top_n.items():
        recom_rate.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = recom_rate[:n]
    
    return top_n

In [45]:
#Defining a fucntion that takes the input of function that returns top_5 products & returns it in a representable dataframe
def top_rec_prod(dict, df):
    for user in dict.keys():
        try:
            for i in range(0,5):
                df.loc[user, 'Product_'+str(i+1)] = dict[user][i]
        except IndexError:
            pass
    return df

In [46]:
#Recommending Top_5 products based of 'SVD' model
svd_recom = top_rec(sur_svd_pred, n=5)
svd_recom_dict = {}
for author, recom in svd_recom.items():
    svd_recom_dict[author] = [product for product, score in recom]
svd_recom_dict

{'Mal': ['Sim Free Motorola Moto G 4th Generation Mobile Phone   Black',
  'TTfone Pluto Big Button Clamshell Flip Unlocked UK Sim Free Mobile Phone   Blue',
  'SAMSUNG Galaxy Ace',
  'Samsung Star II S5260 Smartphone  7 62 cm  3 Zoll  Touchscreen  3MP Kamera  MP3 Player  WLAN  Bluetooth  t9 Trace  onyx black'],
 'lahr2006': ['Apple iPhone 6 16GB',
  'Sony Xperia S',
  'Samsung Galaxy Express I8730',
  'Samsung Galaxy Express I8730',
  'Samsung Galaxy Express I8730'],
 'e bit': ['Samsung Galaxy S6 Edge SM G925 32GB',
  'Samsung Galaxy S6 Edge SM G925 32GB',
  'Samsung Galaxy S6 Edge SM G925 32GB',
  'Samsung Galaxy S6 Edge SM G925 32GB',
  'Samsung Galaxy S6 Edge SM G925 32GB'],
 'Amazon Customer': ['Samsung Galaxy S4 Black   No Contract Phone  U S  Cellular ',
  'Nokia 105 Sim Free Smartphone   Blue',
  'Samsung Galaxy S6 Edge Plus SM G928c 32GB Factory Unlocked White',
  'Samsung Galaxy Note 3 USB 3 0 5 Feet Data Cable  Non Retail Packaging  White',
  'Galaxy Samsung Galaxy Note 4 SM

In [47]:
#Representing the recommended products predicted based on 'SVD' model
svd_recom_df = pd.DataFrame()
top_rec_prod(svd_recom_dict, svd_recom_df)

,Product_1,Product_2,Product_3,Product_4,Product_5
Mal,Sim Free Motorola Moto G 4th Generation Mobile...,TTfone Pluto Big Button Clamshell Flip Unlocke...,SAMSUNG Galaxy Ace,Samsung Star II S5260 Smartphone 7 62 cm 3 Z...,NaN
lahr2006,Apple iPhone 6 16GB,Sony Xperia S,Samsung Galaxy Express I8730,Samsung Galaxy Express I8730,Samsung Galaxy Express I8730
e bit,Samsung Galaxy S6 Edge SM G925 32GB,Samsung Galaxy S6 Edge SM G925 32GB,Samsung Galaxy S6 Edge SM G925 32GB,Samsung Galaxy S6 Edge SM G925 32GB,Samsung Galaxy S6 Edge SM G925 32GB
Amazon Customer,Samsung Galaxy S4 Black No Contract Phone U...,Nokia 105 Sim Free Smartphone Blue,Samsung Galaxy S6 Edge Plus SM G928c 32GB Fact...,Samsung Galaxy Note 3 USB 3 0 5 Feet Data Cabl...,Galaxy Samsung Galaxy Note 4 SM N910T 4G LTE ...
David,Lenovo Motorola Moto G 4G 2 Generazione Smar...,Honor 7 Smartphone 13 2 cm 5 2 Zoll Touchsc...,BLU Tank II T193 Unlocked GSM Dual SIM Cell Ph...,APPLE iPhone 7 Plus Black 128 GB,Motorola Moto G XT1072 2 Generacin 4G Smar...
...,...,...,...,...,...
Pradip,Lenovo Vibe K5 Gold VoLTE update,NaN,NaN,NaN,NaN
Donald L,Huawei Ascend Mate 7 16GB Unlocked GSM 4G LTE ...,NaN,NaN,NaN,NaN
Mehmet,Samsung 936 S5230W Star WiFi,NaN,NaN,NaN,NaN
Mim,Huawei Ascend G7 Smartphone 13 97 cm 5 5 Zol...,NaN,NaN,NaN,NaN


In [48]:
#Recommending Top_5 products based on 'kwmup' model
kwmup_recom = top_rec(kwmup_pred, n=5)
kwmup_recom_dict = {}
for author, recom in kwmup_recom.items():
    kwmup_recom_dict[author] = [product for product, score in recom]
kwmup_recom_dict

{'Mal': ['Sim Free Motorola Moto G 4th Generation Mobile Phone   Black',
  'TTfone Pluto Big Button Clamshell Flip Unlocked UK Sim Free Mobile Phone   Blue',
  'SAMSUNG Galaxy Ace',
  'Samsung Star II S5260 Smartphone  7 62 cm  3 Zoll  Touchscreen  3MP Kamera  MP3 Player  WLAN  Bluetooth  t9 Trace  onyx black'],
 'lahr2006': ['Nokia Asha 311',
  'Apple iPhone 6 16GB',
  'Mercedes Benz Club   Eau de Toilette',
  'Samsung GALAXY S III Mini 4GB',
  'Samsung Galaxy Express I8730'],
 'e bit': ['Smartphone LG G Flex',
  'Samsung Galaxy J1 Ace SM J110L',
  'Sony Xperia Z5',
  'Smartphone LG G Flex',
  'Smartphone BlackBerry Q10'],
 'Amazon Customer': ['Motorola Moto G 3rd Generation DUAL SIM 4G LTE 8GB SIM Free Smartphone   Black',
  'TUDIA Ultra Slim Full Matte ARCH TPU Bumper Protective Case for Asus ZenFone 2 Laser 5 5 Inch  ZE550KL   Not For ZE601KL 6 0 Inch   Gray ',
  'LG G4 Stylus H540 5 7 LCD Unlocked World Phone 8 gb  Titan Gray  INTERNATIONAL VERSION NO WARRANTY',
  'Motorola MOTO Z

In [49]:
#Representing the recommended products predicted based on 'kwmup' model
kwmup_recom_df = pd.DataFrame()
top_rec_prod(kwmup_recom_dict, kwmup_recom_df)

,Product_1,Product_2,Product_3,Product_4,Product_5
Mal,Sim Free Motorola Moto G 4th Generation Mobile...,TTfone Pluto Big Button Clamshell Flip Unlocke...,SAMSUNG Galaxy Ace,Samsung Star II S5260 Smartphone 7 62 cm 3 Z...,NaN
lahr2006,Nokia Asha 311,Apple iPhone 6 16GB,Mercedes Benz Club Eau de Toilette,Samsung GALAXY S III Mini 4GB,Samsung Galaxy Express I8730
e bit,Smartphone LG G Flex,Samsung Galaxy J1 Ace SM J110L,Sony Xperia Z5,Smartphone LG G Flex,Smartphone BlackBerry Q10
Amazon Customer,Motorola Moto G 3rd Generation DUAL SIM 4G LTE...,TUDIA Ultra Slim Full Matte ARCH TPU Bumper Pr...,LG G4 Stylus H540 5 7 LCD Unlocked World Phone...,Motorola MOTO Z PLAY 32GB 5 5 4G LTE Fact...,BlackBerry Q10 SIM Free Smartphone Black
David,Sony Ericsson Xperia neo V Smartphone Touchsc...,Microsoft Nokia Lumia 635 4G UK SIM Free Smart...,Nokia LUMIA 625 NOIR,PRIV by BlackBerry,BlackBerry Curve 9360 Sim Free Smartphone Black
...,...,...,...,...,...
Pradip,Lenovo Vibe K5 Gold VoLTE update,NaN,NaN,NaN,NaN
Donald L,Huawei Ascend Mate 7 16GB Unlocked GSM 4G LTE ...,NaN,NaN,NaN,NaN
Mehmet,Samsung 936 S5230W Star WiFi,NaN,NaN,NaN,NaN
Mim,Huawei Ascend G7 Smartphone 13 97 cm 5 5 Zol...,NaN,NaN,NaN,NaN


In [50]:
#Recommending Top_5 products based on 'kwmip' model
kwmip_recom = top_rec(kwmip_pred, n=5)
kwmip_recom_dict = {}
for author, recom in kwmip_recom.items():
    kwmip_recom_dict[author] = [product for product, score in recom]
kwmip_recom_dict

{'Mal': ['Sim Free Motorola Moto G 4th Generation Mobile Phone   Black',
  'TTfone Pluto Big Button Clamshell Flip Unlocked UK Sim Free Mobile Phone   Blue',
  'SAMSUNG Galaxy Ace',
  'Samsung Star II S5260 Smartphone  7 62 cm  3 Zoll  Touchscreen  3MP Kamera  MP3 Player  WLAN  Bluetooth  t9 Trace  onyx black'],
 'lahr2006': ['Apple iPhone 6 16GB',
  'Samsung GALAXY S III Mini 4GB',
  'Mercedes Benz Club   Eau de Toilette',
  'Nokia Asha 311',
  'Nokia X6 16GB'],
 'e bit': ['Samsung Galaxy J1 Ace SM J110L',
  'Sony Xperia Z5',
  'Samsung Galaxy S6 Edge SM G925 32GB',
  'Sony Xperia Z5 Premium',
  'Samsung Galaxy S6 Edge SM G925 32GB'],
 'Amazon Customer': ['Motorola Moto G 3rd Generation DUAL SIM 4G LTE 8GB SIM Free Smartphone   Black',
  'TUDIA Ultra Slim Full Matte ARCH TPU Bumper Protective Case for Asus ZenFone 2 Laser 5 5 Inch  ZE550KL   Not For ZE601KL 6 0 Inch   Gray ',
  'LG G4 Stylus H540 5 7 LCD Unlocked World Phone 8 gb  Titan Gray  INTERNATIONAL VERSION NO WARRANTY',
  'Sam

In [51]:
#Representing the recommended products predicted based on 'kwmip' model
kwmip_recom_df = pd.DataFrame()
top_rec_prod(kwmip_recom_dict, kwmip_recom_df)

,Product_1,Product_2,Product_3,Product_4,Product_5
Mal,Sim Free Motorola Moto G 4th Generation Mobile...,TTfone Pluto Big Button Clamshell Flip Unlocke...,SAMSUNG Galaxy Ace,Samsung Star II S5260 Smartphone 7 62 cm 3 Z...,NaN
lahr2006,Apple iPhone 6 16GB,Samsung GALAXY S III Mini 4GB,Mercedes Benz Club Eau de Toilette,Nokia Asha 311,Nokia X6 16GB
e bit,Samsung Galaxy J1 Ace SM J110L,Sony Xperia Z5,Samsung Galaxy S6 Edge SM G925 32GB,Sony Xperia Z5 Premium,Samsung Galaxy S6 Edge SM G925 32GB
Amazon Customer,Motorola Moto G 3rd Generation DUAL SIM 4G LTE...,TUDIA Ultra Slim Full Matte ARCH TPU Bumper Pr...,LG G4 Stylus H540 5 7 LCD Unlocked World Phone...,Samsung Galaxy S7 SM G930F 32GB 4G Gold smar...,Apple iPhone 5S 16GB Space Grey Locked to ...
David,Sony Ericsson Xperia neo V Smartphone Touchsc...,Nokia LUMIA 625 NOIR,Samsung Galaxy S3 Neo Smartphone libre Andro...,G4 H815 LG,Motorola Moto G Smartphone Android 114 3 mm...
...,...,...,...,...,...
Pradip,Lenovo Vibe K5 Gold VoLTE update,NaN,NaN,NaN,NaN
Donald L,Huawei Ascend Mate 7 16GB Unlocked GSM 4G LTE ...,NaN,NaN,NaN,NaN
Mehmet,Samsung 936 S5230W Star WiFi,NaN,NaN,NaN,NaN
Mim,Huawei Ascend G7 Smartphone 13 97 cm 5 5 Zol...,NaN,NaN,NaN,NaN


### 9. Try other techniques (Example: cross validation) to get better results. [3 Marks]

In [52]:
#Using Cross-Validation on 'svd' model
sur_svd_cv = cross_validate(sur_svd, mobs, measures=['rmse'], cv=10)
display(sur_svd_cv)

#Getting the average of RMSE for all the folds of 'svd' model
print('Average RMSE of SVD Model on Cross-Valiation is:',sur_svd_cv['test_rmse'].mean())

{'test_rmse': array([2.64722367, 2.63829356, 2.6386901 , 2.65198511, 2.65789164,
        2.63273407, 2.66253884, 2.6434483 , 2.65713242, 2.64655686]),
 'fit_time': (6.417217969894409,
  6.637107610702515,
  6.326216220855713,
  6.284646272659302,
  6.4330174922943115,
  6.452186346054077,
  6.321595191955566,
  6.317474842071533,
  6.700809955596924,
  6.449233531951904),
 'test_time': (0.6076865196228027,
  0.3096158504486084,
  0.6272604465484619,
  0.33748745918273926,
  0.30129027366638184,
  0.29938530921936035,
  0.31386637687683105,
  0.31413841247558594,
  0.31580018997192383,
  0.31465768814086914)}

Average RMSE of SVD Model on Cross-Valiation is: 2.6476494573893232


In [53]:
#Using Corss-Validation of 'kwmup' model
kwmup_cv = cross_validate(kwmup, mobs, measures=['rmse'], cv=10)
display(kwmup_cv)

#Getting the average RMSE for all the folds of 'kwmp' model.
print('Average RMSE of KWMUP Model on Cross-Valiation is:',kwmup_cv['test_rmse'].mean())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

{'test_rmse': array([2.7723857 , 2.73947268, 2.74576061, 2.75315569, 2.75134875,
        2.75603597, 2.75569489, 2.73767373, 2.74177161, 2.73433725]),
 'fit_time': (14.224066972732544,
  13.421332359313965,
  14.209805488586426,
  14.513230562210083,
  12.712089538574219,
  13.439899206161499,
  13.297188758850098,
  12.897126913070679,
  13.207193613052368,
  13.217761278152466),
 'test_time': (8.90055227279663,
  7.601863622665405,
  7.613769292831421,
  8.48350977897644,
  9.42286992073059,
  7.8668413162231445,
  7.426947355270386,
  7.703237771987915,
  7.485613584518433,
  7.363465070724487)}

Average RMSE of KWMUP Model on Cross-Valiation is: 2.7487636880632573


In [54]:
#Using Corss-Validation of 'kwmip' model
kwmip_cv = cross_validate(kwmip, mobs, measures=['rmse'], cv=10)
display(kwmip_cv)

#Getting the average RMSE for all the folds of 'kwmp' model.
print('Average RMSE of KWMIP Model on Cross-Valiation is:',kwmip_cv['test_rmse'].mean())

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

{'test_rmse': array([2.71084945, 2.71037672, 2.70080631, 2.69677351, 2.70912777,
        2.71542663, 2.72502638, 2.70529487, 2.70274288, 2.69455093]),
 'fit_time': (1459.365466117859,
  1362.3217177391052,
  945.8388814926147,
  933.4540104866028,
  1033.2985320091248,
  941.3075799942017,
  1487.4708750247955,
  1441.7643928527832,
  1493.4288313388824,
  1437.9763169288635),
 'test_time': (569.7205376625061,
  373.029714345932,
  358.900084733963,
  362.2257196903229,
  353.1820020675659,
  541.6424822807312,
  534.8917531967163,
  549.1986088752747,
  530.3285183906555,
  377.46287870407104)}

Average RMSE of KWMIP Model on Cross-Valiation is: 2.7070975462292806


In [55]:
#Comparing the RMSE Score of original model with the cross-validation models
#Editing the dataframe created in Que 5. 'Model Accuracy' for comparison purpose
Model_Accuracy.loc['Accuracy(RMSE)_CV',:] = [sur_svd_cv['test_rmse'].mean(), kwmup_cv['test_rmse'].mean(), kwmip_cv['test_rmse'].mean()]
Model_Accuracy

,SVD_Model,UB_KWM_Model,IB_KWM_Model
Accuracy (RMSE),2.641431,2.760045,2.727831
Accuracy(RMSE)_CV,2.647649,2.748764,2.707098


- Here we can see that the RMSE score for SVD_Model is almost same, while it has reduced i.e. improved for User Based KNNWithMeans model & Item Based KNNWithMeans model.

### 10. In what business scenario you should use popularity based Recommendation Systems ? [2 Marks]

- The primary use of popularity based recommendation system is that it doesnot require a user data to recommend products, hence a user who doesnot want to be a member or doesnot want to subscribe, such users can be retained by using popularity based recommendation systems.
- Further, Popularity based Recommendation systems can be used when a new user is introduced to the website.
- It means that when we have no data of user or the product, popularity based recommendation systems are used.
- For eg. if a new user login's for very first time & we have no prefernce as to which products he shall like, we recommened him with the most popular products that other users have used/liked. This way we may increase the chances of retaining a customer.
- Hence the Popularity based Recommendation Systems can be used when personalised recommendations are not required.

### 11. In what business scenario you should use CF based Recommendation Systems ? [2 Marks]

- Collaborative Filter Recommendation systems can be used for existing users or products.
- It means that after using/liking certain product, what are the next products that shall be suggested to a user to retain his interest in website, or if a particular product that has been rate by certain users, how likely it is to be rated & how much by other existing users in our dataset.
- Further, if a user has used few products, we can use CF Filter Recommendations system to predict the ratings that user would provide to other products which he/she has not yet used/liked & based on the predicted ratings we can suggeset user new products.
- While for a product, if a particular product is liked by few users, how that product is likely to be rated by other existing users in our dataset & based on that predicted ratings, the product will be suggested to the users who can use/like it.
- Hence a Collaborative based Recomendation Systems can be suggested for existing user & products, i.e. when the the details of user & prodcuts are available, CF based Recommended System are sugggested.
- For eg. If a customer has already purchased few procucts on Amazon or watched few movies on OTT platform, CF based RS can be used to recommend a user with new product based on his likings & reviews.
- Similarly, if a product is reviewd by certain users, based on the review received by products, it can be recommended to users who have not yet used/liked that product.

### 12. What other possible methods can you think of which can further improve the recommendation for different users ? [2 Marks]

- We came across following Recommendation systems:
    - Popularity Based RS
    - Content Based RS
    - Classification Model Based
    - Collaborative Filtering:
        - User Bsed
        - Itme Based
    - Association Rule Mining/Market Basket Analysis: Apriori Principle
- Further other recommendation systems that can be used to improve recommendations for different users are:
    - Demographic Based RS
    - Utility Based RS
    - Knowledge Based RS
- Futher combinig 2 or more RS can recommend better results. This is called as 'Hybrid Recommendation System'. Since this RS uses uses a combination of multiple RS systems it is better at recommendations & hence is used widely.